In [4]:
import pandas as pd

filename = r"D:\Legal_Document_Summarisation\DATASET\dataset_test.xlsx"

df = pd.read_excel(filename,index_col=0)
df.rename(columns = {'data':'source', 'summary':'target'}, inplace = True)
len(df)

100

In [6]:
from torch.utils.data import Dataset

class LongTextSummarizationDataset(Dataset):
    def __init__(self, data, tokenizer, max_chunk_length, max_summary_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_chunk_length = max_chunk_length
        self.max_summary_length = max_summary_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        document = self.data[idx]['source']
        summary = self.data[idx]['target']

        # Split the document into chunks
        chunks = [document[i:i + self.max_chunk_length] for i in range(0, len(document), self.max_chunk_length)]

        inputs = self.tokenizer(
            chunks,
            max_length=self.max_chunk_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        target = self.tokenizer(
            summary,
            max_length=self.max_summary_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )

        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'labels': target['input_ids'],
        }


In [ ]:
def generate_summary_for_long_text(long_text, model, tokenizer, max_chunk_length, max_summary_length):
    # Split the long text into chunks
    chunks = [long_text[i:i + max_chunk_length] for i in range(0, len(long_text), max_chunk_length)]
    generated_summaries = []

    for chunk in chunks:
        input_ids = tokenizer.encode(chunk, return_tensors="pt", max_length=max_chunk_length, padding="max_length", truncation=True)
        summary_ids = model.generate(input_ids, max_length=max_summary_length, num_beams=4, early_stopping=True)

        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        generated_summaries.append(summary)

    full_summary = " ".join(generated_summaries)
    return full_summary


In [12]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertLMHeadModel
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer

# Load BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertLMHeadModel.from_pretrained(model_name)

def summarize_long_text(long_text, max_chunk_length=125, max_summary_length=240, use_bert=True):
    # Split the long text into smaller chunks
    text_chunks = [long_text[i:i + max_chunk_length] for i in range(0, len(long_text), max_chunk_length)]
    summaries = []

    if use_bert:
        # Generate summaries using BERT-based model for each chunk
        for chunk in text_chunks:
            inputs = tokenizer(chunk, return_tensors="pt", max_length=max_chunk_length, padding="max_length", truncation=True)
            summary_ids = model.generate(inputs["input_ids"], max_length=max_summary_length, num_beams=4, early_stopping=True)
            summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)
    else:
        # Alternatively, you can use extractive summarization with LexRank
        # Make sure to install the sumy library: pip install sumy
        parser = PlaintextParser.from_string(long_text, Tokenizer("english"))
        summarizer = LexRankSummarizer(Stemmer("english"))
        summarizer.stop_words = get_stop_words("english")
        
        for sentence in summarizer(parser.document, 3):  # Adjust the number of sentences in the summary
            summaries.append(str(sentence))

    # Combine the summaries from each chunk into a single summary
    full_summary = " ".join(summaries)
    return full_summary

# Example usage
long_text = """Civil Appeal Nos.3677 3680 of 1986 Etc.
From the Judgment and Order dated 10.3.1986 of the Karnataka High Court in W.P. Nos.
4053 to 4056 of 1985.
Shanti Bhushan, Dr. Y.S. Chitale, H.B. Datar, K.R. Nagaraja, 1011 R.S. Hegde, R.B. Datar, S.S. Jawali, B.P. Singh, N.D.B. Raju, R.P., Wadhwani, Aruneshwar Gupta, Swaraj Kaushal, KMM Khan, S.R. Setia, A.T.M. Sampath and C.S. Vaidyanathan for the appearing parties.
The Judgment of the Court was delivered by CHINNAPPA REDDY, J.
Special leave granted in all the cases.
These appeals raise common questions of law and may therefore, be disposed of by a common judgment.
In exercise of its powers under section 63(7) of the , the Central Government specified 50 as the number of 'All India Tourist Vehicle Permits ' which may be granted by the Karnataka State Transport Authority.
By section 24 of Amending Act 47 of 1978, a proviso to section 63(7) was introduced.
We are concerned in these appeals with the vires and interpretation of this proviso.
By the time the proviso came into force, 36 permits had been granted by the Karnataka State Transport Authority and 14 remained to be granted.
There were as many as 495 applications for the grant of these 14 permits.
By an order dated February 1, 1984, the Supreme Court directed the Karnataka State Transport Authority to dispose of these applications.
The State Transport Authority, by its proceedings dated April 30, 1984, granted 11 out of the 14 permits to the Karnataka State Tourism Development Corporation, one permit to the Indian Tourism Development Corporation and two permits to the Karnataka State Road Transport Corporation.
A number of appeals were preferred to the Karnataka State Transport Appellate Tribunal.
The Tribunal by its order dated February 28, 1985 set aside the grant of the two permits in favour of the Karnataka State Road Transport Corporation, set aside the grant of three out of eleven permits to the Karnataka State Tourism Development Corporation and instead granted three permits to private operators and increased the number of permits granted to the Indian Tourism Development Corporation from one to three.
The Tribunal took the view that having regard to the rule of preference enunciated by the proviso to section 63(7), the applications from the 'non preferred ' category had to be excluded as the number of applications from the applicants who were required to be given preference exceeded the number of permits to be granted.
It was on that ground that the grant of two permits to the Karnataka State Road Transport Corporation was set aside, though the Appellate Tribunal had no doubt regarding the resources and ability of that corporation to operate the tourist services.
It was on that ground again, it was so stated by the Tribunal, that some of the appellants 1012 before the Tribunal had to be denied the grant of permits though otherwise they would have been entitled to the grant of permits having regard to their expertise, experience and resources.
The Tribunal rejected their appeals regretfully.
A large number of applicants filed writ petitions in the High Court.
The writ petitions were rejected by the High Court on the ground that the preference contemplated by the proviso to section 63(7) contemplated exclusion of the 'non preferred ' class if sufficient number of applicants from the preferred classes were available.
The decision of this Court in Sher Singh vs Union of India, ; was distinguished on the ground that in that case the court interpreted the word 'preference ' occurring in section 47(1 H) in the back ground of the provisions of Chapters IV and IV A of the Act, under the former of which the State Transport Undertaking would have preference whereas under the latter the State Transport Undertaking would have a monopoly.
The Karnataka State Road Transport Corporation, the Karnataka State Tourism Development Corporation and some other private operators have filed these appeals by special leave of this Court under article 136 of the Constitution.
Shri Shanti Bhushan, learned counsel for the Karnataka State Road Transport Corporation, Dr. Chitley, learned counsel for some of the private operators, Shri Datar, learned counsel for the Karnataka State Tourism Development Corporation, Shri Sampat and Shri Javali, learned counsel for other private operators submitted that the State Transport Appellate Tribunal and the High Court were wrong in distinguishing the decision of this court in Sher Singh 's case and that the true position was that on a correct interpretation of the proviso to sec.
63(7), the preference became operative only if other things were equal.
It was also urged that the fourth sub clause of the proviso offended article 14 and had to be struck down.
Shri C.S. Vaidyanathan, learned counsel for some of the preferred private operators urged that the view taken by the High Court and the State Transport Appellate Tribunal was correct and that the fourth sub clause of the proviso to section 63(7) did not offend article 14 of the Constitution.
We may now glance at some of the relevant provisions of the . section 2(33) defines a "transport vehicle" as meaning a "public service vehicle or a goods vehicle".
A "public service vehicle" is defined in section 2(25) as "any motor vehicle used or adapted to be used for the carriage of passengers for hire or reward, and includes a motor cab, contract carriage, and stage carriage".
A "motor car" is defined in section 2(16) as "any motor vehicle other than a transport vehicle, 1013 omni bus, road roller, tractor, motor cycle or invalid carriage".
A "contract carriage" is defined as, broadly, a motor vehicle which carries a passenger or passengers for hire or reward under a contract. "Tourist vehicle" is defined by section 2(29 A) as "a contract carriage constructed or adapted and equipped and maintained in accordance with such specifications as the State Government may, by notification in the Official Gazette, specify in this behalf".
"Stage carriage" is defined by section 2(29) as "a motor vehicle carrying or adapted to carry more than six persons excluding the driver which carries passengers for hire or reward at separate fares paid by or for individual passengers, either for the whole journey or for stages of the journey." Chapter IV, sections 42 to 68, deals with "Control of Transport vehicles".
Section 42 prescribes permits for the use of a transport vehicle in any public place.
Sections 46, 47 and 48 deal with the grant of stage carriage permits.
Section 47(1) prescribes the matters to be taken into consideration in granting stage carriage permits and the first consideration, naturally, is "the interest of the public generally.
" The proviso to section 47(1) prescribes that, other things being equal, a registered cooperative society and a person possessing a licence for driving transport vehicles shall be given preference over individual owners in granting stage carriage permits.
Section 47(1 A) enables the State Government to reserve a certain percentage of stage carriage permits for the Scheduled Castes and Scheduled Tribes.
Section 47(1 C) enables the State Government to reserve a certain percentage of stage carriage permits to persons belonging to economically weaker sections of the community.
Section 47(1 H) prescribes that notwithstanding anything contained in the section, an application for stage carriage permit from a State Transport undertaking for operating in any inter State route shall be given preference overall other applications, provided, of course, the authority is satisfied that the State Transport Undertaking would be able to operate in the inter State route without detriment to its responsibility for providing efficient and adequate road transport services in any notified area or notified route.
Sections 49, 50, 51 deal with the grant of contract carriage permits.
Section 52 and 53 deal with private carrier 's permit and sections 54, 55 and 56 deal with public carrier 's permit.
Section 57 deals, generally with the procedure to be followed in applying for and granting permits.
Section 63(1) stipulates, broadly, that a permit granted by the Regional Transport Authority of any one region shall not be valid in any other region, unless the permit has been countersigned by the Regional Transport Authority of that other region, and a permit granted in any one State shall not be valid in any other State unless countersigned by the State Transport Authority of 1014 that other State or by the Regional Transport Authority concerned.
Section 63(7) is the provision with whose interpretation and vires we are primarily concerned in this case.
It is as follows: "(7) Notwithstanding anything contained in sub section(1) but subject to any rules that may be made under this Act, any State Transport Authority may, for the purpose of promoting tourism, grant permits valid for the whole or any part of India, in respect of such number of tourist vehicles as the Central Government may, in respect of that State, specify in this behalf, and the provisions of Sections 49, 50, 51, 57, 58, 59, 59 A, 60, 61 and 64 shall, as far as may be, apply in relation to such permits: Provided that preference shall be given to applications for permits from (i) the India Tourism Development Corporation; (ii) a State Tourism Development Corporation; (iii) a State Tourist Department; (iv) such operators of tourist cars, or such travel agents, as may be approved in this behalf by the Ministry of the Central Government dealing in tourism.
" Section 68 enables the State Government to make rules for the purpose of carrying into effect the provisions of Chapter IV.
Chapter IV A relates to "special provisions relating to State Transport Under takings.
" Sections 68 C, 68 D and 68 E provide for the preparation and publication of schemes of road transport service to be provided by State Transport Undertakings, the procedure to be followed, etc.
Section 68 F(1) prescribes that where, in pursuance of an approved scheme, a State Transport Undertaking applies for a stage carriage permit, a contract carriage permit or a public carrier 's permit in respect of a notified area or notified route, such permit shall be granted to the State Transport Undertaking by the State Transport Authority in a case where the said area or route lies in more than one region and the Regional Transport Authority in any other case.
This is to be so notwithstanding anything to the contrary contained in Chapter IV.
1015 The general scheme of the Act in the matter of grant of permits for stage carriages and contract carriages appears to be that except in the case of a notified route or notified area, where under section 68 F(1) the permit has to be necessarily granted to the State Transport Undertaking, in all other cases, the claims of all eligible applicants must be considered on merits, applying the rules of preference wherever the claims are approximately equal.
Except in the case of a notified route or notified area, the application of no applicant may be altogether excluded from consideration on the sole ground that another applicant is entitled to preference under one or the other provisions of the statute.
The proviso to section 47(1) for example, provides that other conditions being equal, an application for a stage carriage permit from a cooperative society or a person holding a valid licence for driving transport vehicles shall as far as may be, be given preference over applications from individual owners.
There is no problem here since the proviso itself says that the rules of preference will apply only if other conditions are equal.
Section 47(1 H) also enunciates a rules of preference and says that an application for stage carriage permit from State Transport Undertaking for operating in any inter State route shall be given preference overall other applications.
While it is true that section 47(1 H) does not expressly refer to "other things being equal", it appears to be implicit in the provision that other things are equal.
The rule is a rule of preference and not a rule of exclusion.
Section 47(1 H) does not say, for example, like section 68 F(1) that the permit shall be granted to the State Transport Undertaking.
That is how section 47(1 H) was interpreted in Sher Singh 's case.
Desai, J., speaking for the court observed: "However, when an application for a permit is made under Chapter IV, the Undertaking has to compete with private operators who may as well make an application for permit.
When the Undertaking applies for permit under Chapter IV, it must satisfy the Regional Transport Authority that it is better suited than the private operator to render transport facility to the travelling public.
47(1 H) however, provides that in the case of inter State route, the Undertaking will have preference in the matter of stage carriage permit.
Does preference of this nature deny equality guaranteed by article 14? The expression 'preference ' amongst others means prior right, advantage, precedence etc.
But how would it be possible to give precedence to one over the other.
It signifies that other things being equal, 1016 one will have preference over the others.
When an application for a stage carriage permit is being processed as required by sec.
47, the application of the Undertaking for an inter State route shall be examined as application of any other private operator.
Their merits and demerits must be ascertained keeping in view the requirements of (a) to (f) of section 47(1) and after comparing the merits and demerits of both, not with the yard stick of mathematical accuracy, but other things being equal, the application of the Undertaking will have preference over others.
Qualitative and quantitative comparison on broad features of passenger transport facility such as fleet, facilities to travelling public and other relevant consideration may be undertaken and after balancing these factors other things being equal, the application of the Undertaking shall be given preference over other applicants.
There is no question of eliminating private operators merely because the Undertaking applies for a stage carriage permit under Chapter IV.
That situation is catered to under Chapter IV A.
In an application under Chapter IV, Corporation has to enter the arena like any other applicant, face the competition and come up to the level of other private operators intending to obtain stage carriage permits and then in respect of the route in question claim preference.
Would this statutory provision violate equality guaranteed by article 14? The answer is obviously in the negative." . . . . . . . . . . . . . . . . . ". . . let it be made clear that while considering the application for stage carriage permit under section 47, the private operator has an equal chance to get a permit even on inter State route if it shows that the Undertaking is either unable to provide efficient and economical service or that the private operator is better equipped to render the same.
Preference in this context would mean that other things generally appearing to be qualitatively and quantitatively equal though not with mathematical accuracy, statutory provision will tilt the balance in favour of the Undertaking.
" What has been said by the Court with reference to the preference 1017 provided for in section 47(1 H) applies with equal force to the preference provided for by the proviso to section 63(7).
In the judgment under appeal, the High Court attempted to distinguish the decision of this Court in Sher Singh 's case on the ground that any other interpretation would have wiped out the difference between Chapter IV and Chapter IV A.
We do not think the High Court was right in distinguishing the case in that fashion.
The reference to Chapter IV A there was for the purpose of contrasting the exclusion contemplated by section 68.F(1) with the preference to be given under section 47 (1 H) and so to interpret the word 'preference ' occurring in section 47(1 H).
We have no hesitation in saying that all that has been said about 'preference ' in Sher Singh 's case in relation to section 47(1 H) applies mutatis mutandis to the preference contemplated by the proviso to section 63(7).
Since the State Transport Appellate Tribunal and the High Court have failed to consider the merits of the claims of the Karnataka State Road Transport Corporation and the private operators who did not get a certificate of approval from the Central Government, because of the rule of preference contained in proviso to section 63(7), the proper course for us is to set aside the orders of the State Transport Appellate Tribunal and the High Court and to direct the State Transport Appellate Tribunal to re hear the appeals and dispose them of in accordance with law, after considering the claims of the eligible applicants in the manner indicated in Sher Singh 's case and now.
A question was raised before us about the vires of the fourth clause of the proviso to section 63(7).
Clauses (i) to (iii) of the proviso providing for preference to be given to applications for permits from the Indian Tourism Development Corporation, the State Tourism Development Corporation and the State Tourist Department were not questioned, but the preference provided for by clause (iv) and to be given to "such operators of tourist cars, or such travel agents, as may be approved in this behalf by the Ministry dealing in tourism" was questioned as an infringement of article 14 of the Constitution.
We find it difficult to sustain this clause and uphold its validity.
The very idea that a Tribunal created by a statute for the purpose of considering rival claims and granting permits on merits should be compelled to give peference to persons securing the approval of the executive Government, appears to us to be arbitrary and unreasonable.
To the extent that it goes, the clause pre empts the decision of the designated tribunal by executive discretion.
It was said that the clause contained sufficient guidelines for the exercise of discretion in granting approval by the Central Government.
It was said that the object of the proviso to section 1018 63(7) was very obviously the promotion of tourism and the approval of the Central Government would be given to those operators of tourist cars and travel agents who may be expected to serve that purpose.
It is difficult to agree with these submissions.
In the first place, clause (d), it is seen, provides for a preference, not to operators of tourist vehicles but to operators of tourist cars and travel agents.
Though the permits to be granted are for tourist vehicles, the preference is confined to operators of tourist cars and travel agents.
One may understand a preference granted to operators of tourist vehicles but it is difficult to understand why preference should be given to operators of tourist cars in the matter of granting permits for tourist vehicles which may well be omnibuses required to travel long distances.
Surely it cannot be said that experience of running a tourist taxi is a better qualification than running a tourist bus when the question of granting permits for tourist vehicles arises.
The High Court of Karnataka tided over the difficulty by interpreting the expression "tourist cars" as meaning "tourist vehicles".
It is difficult to agree with the interpretation of the Karnataka High Court having regard to the definitions of "Transport Vehicle" and "motor car" contained in section 2(29 A) and section 2(16) respectively.
While a tourist vehicle may include a motor car, a motor car, by definition, excludes an omnibus.
In the second place, we have no indication as to the manner in which the approval of the Central Government is to be sought and granted and the considerations which are expected to weigh with the Central Government.
Shri C.S. Vaidyanathan, learned counsel for the 'Preferred ' Operators has placed before us 'a Scheme for granting approval to tourist transport operators in India. ' The scheme makes no reference to the proviso to sec.
63(7) of the .
On the other hand, it mentions that approval carries with it certain privileges, such as, allotment of ex STC vehicles and that it is, therefore, necessary that the department is able to exercise some control on the functioning of these operators.
The terms and conditions to be fulfilled are that 'the party must have been in the car hire business for a period of 2 years with the help of cars of indigenous make or cars obtained from elsewhere and should have sufficient contacts with travel agencies hoteliers/airlines, etc.
and should be financially sound or that they should be owning and operating five vehicles as tourist taxies of either indigenous make or acquired from elsewhere regardless of the period in the car hire business or that they should be ex Defence Service personnel, who satisfy certain prescribed conditions.
It is seen that the scheme excludes omnibus operators and requires applicants to have 1019 either two years ' experience in the car hire business with contacts in the tourist business or to own five tourist taxies.
There are no guidelines as to how the discretion to grant approval is to be exercised once the minimum conditions are fulfilled.
The matter appears to be left to the total discretion of the Central Government, virtually as we said, pre empting the decision of the statutory tribunal.
We think that clause (iv) of the proviso to section 63(7) is unconstitutional and we so declare it.
In the result we set aside the orders of the State Transport Appellate Tribunal and the High Court and direct the State Transport Appellate Tribunal to hear the appeals in the light of what we have said.
P.S.S. Appeals allowed.
"""
final_summary = summarize_long_text(long_text, use_bert=True)
print(final_summary)


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-paddi

KeyboardInterrupt: 